# Prise en charge des pneumopathies aigues communautaires

Ce notebook vise à établir un POC d'un outil qui pourrait être mis à disposition de professionnels de santé.  
**Fonctionnalité principale** : interface conversationnelle de questions-réponses basé sur un LLM qui peut répondre à des questions portant sur un document   
**Utilisateurs principaux** : professionnels de santé  
**Workflow** : l'utilisateur charge le document, l'outil analyse le document, l'utilisateur pose des questions sur le document, l'outil répond en fonction des données du document  
**Choix techniques** : l'outil proposé consiste en un LLM déjà entraîné, augmenté par le document fournit via un process RAG - Retrieval Augmented Generation.

## Environment setup

In [ ]:
# Prerequisites
!pip install python-docx mistralai faiss-cpu

In [2]:
# Imports
import docx
import textwrap
from mistralai import Mistral
import os
import numpy as np
import time
import faiss

In [3]:
# Mistral API Client
api_key= os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

## Développement de l'outil et du modèle

### Traitement du document, extraction des informations

Le document consiste en des données textuelles à extraire : paragraphes, tableaux, images.  
NB : le cas de l'image ne sera pas traité dans un premier temps, on pourrait utiliser une librairie d'OCR pour en extraire des informations brutes, ou un LLM pour structurer les informations extraites (ex. image-to-text)

In [5]:
document = docx.Document("data/Prise en charge des Pneumopathies aigues communautaires V2.docx")

In [6]:
def extract_data(document):

    doc_chunks = []

    # Extraction des textes dans l'ordre du document
    for item in document.iter_inner_content():

        # Paragraphes
        if isinstance(item, docx.text.paragraph.Paragraph):
            doc_chunks.append(item.text.strip())

        # Tables
        elif isinstance(item, docx.table.Table):
            for row in item.rows:
                row_data = ": ".join([cell.text for cell in row.cells])
                doc_chunks.append(row_data)

    # Nettoyage des éléments vides
    doc_chunks = [chunk for chunk in doc_chunks if chunk != ""]

    # Fusion des éléments
    doc_text = "\n".join(doc_chunks)

    return doc_text

In [7]:
doc_text = extract_data(document)

### Construction du RAG

In [8]:
# Découpage du document en chunks
def split_text(document, chunk_size=300):
    paragraphs = document.split("\n")
    chunks = []
    for paragraph in paragraphs:
        chunks.extend(textwrap.wrap(paragraph, chunk_size))
    return chunks

In [9]:
chunks = split_text(doc_text)

In [11]:
# Génération d'un embedding pour un chunk
def get_one_embedding(chunk):
    embedding = client.embeddings.create(
        model = "mistral-embed",
        inputs = chunk
    )

    return embedding.data[0].embedding

In [12]:
# Limite d'utilisation API Mistral atteinte (vesion gratuite) : découpage en plusieurs parties pour générer les embeddings du doc d'entrée
# Warning - solution non scalable si documents plus longs : considérer un plan payant pour accès API ou gérer l'embedding en local

def get_all_embeddings(chunks, split_size=20):
    embeddings = []
    split_size = 30
    number_of_splits = len(chunks) // split_size + 1

    for i in range(number_of_splits):
        print("Chunk n°",i,"/",number_of_splits-1)
        split_of_chunks = chunks[split_size*i:min(split_size*(i+1),len(chunks))]
        embeddings_of_split = [get_one_embedding(chunk) for chunk in split_of_chunks]
        embeddings.extend(embeddings_of_split)

        time.sleep(60) # Attente pour limite API

    embeddings = np.array(embeddings)

    return embeddings

In [13]:
embeddings = get_all_embeddings(chunks)

Chunk n° 0 / 5
Chunk n° 1 / 5
Chunk n° 2 / 5
Chunk n° 3 / 5
Chunk n° 4 / 5
Chunk n° 5 / 5


In [14]:
# Charger les embeddings du document dans une vector database FAISS
def create_index_from_embeddings(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    return index

In [15]:
index = create_index_from_embeddings(embeddings)

Le LLM avec RAG est prêt pour être utilisé avec une question utilisateur !

### Générer une réponse augmentée du document

In [16]:
# L'utilisateur pose sa question
def get_user_prompt():
    question = input("Une question basée sur le document chargé : ")
    return question

In [17]:
question = get_user_prompt()

In [18]:
question

'Quels sont les critères majeurs de diagnostic de pneumopathie aigue grave ?'

In [19]:
# Création de l'embedding associé à la question
question_embedding = np.array([get_one_embedding(question)])

In [20]:
# Retrouver les chunks les plus similaires dans l'index
def retrieve_chunk(index, question_embedding, top_k = 5):
    D, I = index.search(question_embedding, k=top_k)
    ind_chunks = I.tolist()[0]
    retrieved_chunks = [chunks[i] for i in range(len(chunks)) if i in ind_chunks]
    return " ".join(retrieved_chunks)

In [21]:
retrieved_chunk = retrieve_chunk(index, question_embedding)

In [49]:
# Combiner la question et la réponse dans un même prompt
def create_prompt(user_prompt, retrieved_chunk):
    prompt = f"""
    Contexte : {retrieved_chunk}
    -----------------------------------------
    Étant donné le context fourni ci-dessus, et sans utiliser de connaissance préalable, répond à la question suivante
    -----------------------------------------
    Question : {user_prompt}
    -----------------------------------------
    Format attendu : réponse complète
    -----------------------------------------
    Réponse :
    """

    return prompt

In [23]:
prompt = create_prompt(question, retrieved_chunk)

In [50]:
# Génération de la réponse par envoi du prompt au modèle decoder-only Mistral
def query_mistral(prompt, model="open-mistral-7b"):
    messages = [{
        "role": "user",
        "content": prompt
    }]

    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )

    return chat_response.choices[0].message.content

In [25]:
response = query_mistral(prompt)

In [26]:
print(response)

Les critères majeurs de diagnostic de pneumopathie aigue grave sont : Détresse respiratoire nécessitant recours à la ventilation mécanique.

Ceci est tiré du contexte fourni où il est précisé qu'un des critères majeurs de diagnostic de pneumopathie aigue grave est la nécessité de recours à la ventilation mécanique.


## Full pipeline

Fonction bout-en-bout pour un utilisateur qui a déjà chargé et fait analysé son document :    
- Input de la question de l'utilisateur
- Génération de l'embedding et recherche des chunks les plus simmilaires dans l'index
- Query au chat Mistral et affichage de la réponse

In [34]:
def ask_mistral_augmented(index, model="open-mistral-7b"):
    question = get_user_prompt()
    question_embedding = np.array([get_one_embedding(question)])
    retrieved_chunk = retrieve_chunk(index, question_embedding)
    prompt = create_prompt(question, retrieved_chunk)
    return question, query_mistral(prompt)

### Quelques exemples

In [53]:
question, response = ask_mistral_augmented(index)
print("Question :",question)
print("Réponse :",response)

Question : Quelles sont les recommandations d'antibiotiques pour un adulte atteint d'une PAC non grave sans commorbidités ?
Réponse : Les recommandations d'antibiotiques pour un adulte atteint d'une Pneumonie non grave sans comorbidités sont l'Amoxicilline à une dose de 1-2g par 8 heures par voie orale ou intraveineuse, comme indiqué dans les recommandations thérapeutiques de la Commission des Médicaments Anti-Infectieux (COMAI) réalisées et validées le 20/06/2025.


In [52]:
question, response = ask_mistral_augmented(index)
print("Question :",question)
print("Réponse :",response)

Question : Quels examens ne sont pas recommandés pour une PAC et pourquoi ?
Réponse : Les examens de dosage de la CRP (C-réactive protéine) et PCT (Procalcitonine) ne sont pas systématiquement recommandés pour un diagnostic et/ou suivi de pneumonie aiguë (PAC). Cela est dû au fait qu'ils ne sont pas spécifiques pour la détection de la bactérie comme cause de la PAC et peuvent être altérés par d'autres processus inflammatoires ou infections non respiratoires. Dans le contexte de PAC graves, un diagnostic différentiel doit être effectué et les examens doivent être choisis en fonction de la suspicion clinique, du contexte de l'hospitalisation et de la gravité de la maladie.


In [54]:
question, response = ask_mistral_augmented(index)
print("Question :",question)
print("Réponse :",response)

Question : Dans quels cas puis-je prescrire de l'amoxicilline-acide clavulanique ?
Réponse : L'amoxicilline-acide clavulanique peut être prescrite en cas de comorbidité, car elle possède une activité antibiotique plus large que l'amoxicilline seule, en particulier contre les bactéries productrices d'enzymes β-lactamases résistantes aux β-lactamines. Par ailleurs, elle est également utilisée dans le traitement de la pneumopathie d'inhalation, pour sa bonne pénétration pulmonaire.
